In [107]:
import pandas as pd
import numpy as np
import re

In [5]:
df = pd.read_csv("D:/Project_DA/EDA_of_Airline_dataset/Dataset/airlines_data_csv.csv")

In [6]:
df.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR ? DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU ? IXR ? BBI ? BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL ? LKO ? BOM ? COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU ? NAG ? BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR ? NAG ? DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [8]:
df.shape

(10683, 11)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


### *Data Cleaning and Preprocessing*

In [20]:
#display the row , where total stop is nan, so that we can replace that nan value
df[df['Total_Stops'].isnull()]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,6/05/2019,Delhi,Cochin,NaN,09:45,09:25 07 May,23h 40m,NaN,No info,7480


Missing value of "Route" and "Total stops" belongs to Air India Airline

In [21]:
df.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

We have noticed each airline is subdivided into two different parts

In [205]:
#working with the copy of the original dataset
df_A = df.copy()

In [134]:
#We will combine the 'two-parts' airlines to make our categorical features more consistent with the rest of the variables
df_A['Airline'] = np.where(df_A['Airline'] == "Jet Airways Business", "Jet Airways", df_A['Airline'])
df_A['Airline'] = np.where(df_A['Airline'] == "Multiple carriers Premium economy", "Multiple carriers", df_A['Airline'])
df_A['Airline'] = np.where(df_A['Airline'] == "Vistara Premium economy", "Vistara", df_A['Airline'])

In [206]:
#converting "Total_stops" into a numeric column
df_A.Total_Stops.unique().tolist()

['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops']

In [207]:
df_A.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)

C:\Users\NAYANIKA\AppData\Local\Temp\ipykernel_7684\492365852.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_A.replace({"non-stop": 0, "1 stop": 1, "2 stops": 2, "3 stops": 3, "4 stops": 4}, inplace = True)


In [208]:
df_A.groupby('Airline')['Total_Stops'].agg(pd.Series.mode).to_frame()

,Total_Stops
Airline,
Air Asia,0.0
Air India,2.0
GoAir,1.0
IndiGo,0.0
Jet Airways,1.0
Jet Airways Business,1.0
Multiple carriers,1.0
Multiple carriers Premium economy,1.0
SpiceJet,0.0


In [209]:
#replacing the nan value of Air india for "Total_stops" with the most frequent total stops
df_A['Total_Stops'].replace(np.nan , 2, inplace = True)

C:\Users\NAYANIKA\AppData\Local\Temp\ipykernel_7684\2687061901.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_A['Total_Stops'].replace(np.nan , 2, inplace = True)


#### Feature Transformation

In [210]:
#Converting 'Date_of_Journey', 'Dep_Time', and 'Arrival_Time' to datetime format
df_A['Date_of_Journey'] = pd.to_datetime(df_A['Date_of_Journey'], format = "%d/%m/%Y")


In [211]:
# Combining 'Date_of_Journey' with 'Dep_Time' (since Dep_Time is time-only)
df_A['Dep_Time'] =pd.to_datetime(df_A['Date_of_Journey'].astype(str) + ' ' + df_A['Dep_Time'])

In [212]:
df_A['Dep_Hour'] = df_A['Dep_Time'].dt.hour

In [213]:
# Function to handle mixed date and time formats in 'Arrival_Time'
def convert_arrival_time(arrival, journey_date):
    # Check if arrival contains a date (using regex)
    if re.search(r'\d{2} \w{3}', arrival):
        # If it contains a date, parse and set the correct year from 'Date_of_Journey'
        parsed_time = pd.to_datetime(arrival, format='%H:%M %d %b', errors='coerce')
        return parsed_time.replace(year=journey_date.year)  # Set the correct year from 'Date_of_Journey'
    else:
        # If it only contains time, combine it with the 'Date_of_Journey'
        return pd.to_datetime(journey_date.strftime('%Y-%m-%d') + ' ' + arrival, errors='coerce')

In [214]:
# Apply the function to convert 'Arrival_Time'
df_A['Arrival_Time'] = df_A.apply(lambda row: convert_arrival_time(row['Arrival_Time'], row['Date_of_Journey']), axis=1)

In [215]:
df_A['Arrival_Hour'] = df_A['Arrival_Time'].dt.hour

In [216]:
df_A['Duration_Day_Time'] = df_A['Arrival_Time'] - df_A['Dep_Time']

In [217]:
#taking only hour, minute and seconds from duration_day_time, and converting them datetime format for easier analysis
df_A['Duration_Time'] = pd.to_datetime(df_A['Duration_Day_Time'].apply(lambda x: f"{x.components.hours:02}:{x.components.minutes:02}:{x.components.seconds:02}"))

C:\Users\NAYANIKA\AppData\Local\Temp\ipykernel_7684\1103671046.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_A['Duration_Time'] = pd.to_datetime(df_A['Duration_Day_Time'].apply(lambda x: f"{x.components.hours:02}:{x.components.minutes:02}:{x.components.seconds:02}"))


In [218]:
df_A['Duration_HoursInMins']=(df_A['Duration_Time'].dt.hour)*60

In [219]:
df_A['Duration_TotalInMins'] = (df_A['Duration_Time'].dt.minute)+df_A['Duration_HoursInMins']

#### Splitting 'Departure/Arrival_Time' into Time Zones

In [220]:
df_A['Dep_timezone'] = pd.cut(df_A.Dep_Hour, [0,6,12,18,24], labels=['Night','Morning','Afternoon','Evening'])

In [221]:
df_A['Arrival_timezone'] = pd.cut(df_A.Arrival_Hour, [0,6,12,18,24], labels=['Night','Morning','Afternoon','Evening'])

#### Categorical Encoding

In [222]:
#One_hot_encoding
dummies = pd.get_dummies(df_A[['Airline', 'Source', 'Destination']], prefix=['Airline', 'Source', 'Destination'])

In [224]:
df_encoded = pd.concat([df_A, dummies], axis=1)

In [226]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 42 columns):
 #   Column                                     Non-Null Count  Dtype          
---  ------                                     --------------  -----          
 0   Airline                                    10683 non-null  object         
 1   Date_of_Journey                            10683 non-null  datetime64[ns] 
 2   Source                                     10683 non-null  object         
 3   Destination                                10683 non-null  object         
 4   Route                                      10682 non-null  object         
 5   Dep_Time                                   10683 non-null  datetime64[ns] 
 6   Arrival_Time                               10683 non-null  datetime64[ns] 
 7   Duration                                   10683 non-null  object         
 8   Total_Stops                                10683 non-null  float64        
 9   Additi